# Clase Interactiva: Intervalos de Confianza para la Media

Bienvenidos a esta sesión interactiva en la que aprenderemos a **calcular** e **interpretar** intervalos de confianza (IC) para la media poblacional a partir de una muestra.

**Objetivos de la clase:**
- Comprender qué es un intervalo de confianza y su importancia en la toma de decisiones.
- Calcular el IC para la media en dos casos:
  - Cuando la desviación estándar poblacional ($\sigma$) es conocida.
  - Cuando la desviación estándar se estima a partir de la muestra (utilizando la distribución t de Student).
- Explorar cómo influyen el tamaño de la muestra, el nivel de confianza y la variabilidad en el ancho del intervalo.
- Aplicar estos conceptos de forma interactiva mediante gráficos y ejercicios.


## Teoría Básica sobre Intervalos de Confianza para la Media

Un **intervalo de confianza** es un rango de valores basado en datos muestrales que, con un determinado nivel de confianza, se espera que contenga el parámetro poblacional (en este caso, la media $\mu$).

### Caso 1: $\sigma$ Conocido

- **Error Estándar (SE):**
  
  $
    SE = \frac{\sigma}{\sqrt{n}}
  $

- **Fórmula del Intervalo de Confianza al $(100(1-\alpha))$%:**

  $
    \bar{X} \pm Z_{\alpha/2} \frac{\sigma}{\sqrt{n}}
  $

  Donde $Z_{\alpha/2}$ es el valor crítico obtenido de la distribución normal.

### Caso 2: $\sigma$ Desconocido

- **Error Estándar (SE):**
  
  $
    SE = \frac{s}{\sqrt{n}}
  $

- **Fórmula del Intervalo de Confianza al $(100(1- \alpha))$%:**

  $
    \bar{X} \pm t_{\alpha/2,\, n-1} \frac{s}{\sqrt{n}}
  $

  Donde ($t_{\alpha/2,\, n-1}$) es el valor crítico de la distribución t de Student con $(n-1)$ grados de libertad.


In [3]:
# Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, t
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Configuración para visualizar las gráficas directamente en el Notebook
%matplotlib inline

def calculate_ci(mean, error, alpha, ci_type="known", n=None):
    """
    Calcula el intervalo de confianza para la media.
    
    Parámetros:
      - mean: Media muestral.
      - error: Error estándar (σ/√n o s/√n).
      - alpha: Nivel de significancia (por ejemplo, 1 - 0.95 para un 95%).
      - ci_type: 'known' si σ es conocido; 'unknown' si se usa s.
      - n: Tamaño de la muestra (requerido para el caso t).
    
    Retorna:
      - lower: Límite inferior del intervalo.
      - upper: Límite superior del intervalo.
      - cv: Valor crítico empleado.
    """
    if ci_type == "known":
         cv = norm.ppf(1 - alpha/2)
         lower = mean - cv * error
         upper = mean + cv * error
    else:
         cv = t.ppf(1 - alpha/2, df=n-1)
         lower = mean - cv * error
         upper = mean + cv * error
    return lower, upper, cv

def plot_ci(mean, error, lower, upper, cv, ci_type="known", n=None):
    """
    Grafica la función de densidad (normal o t) y resalta el intervalo de confianza.
    """
    x = np.linspace(mean - 4 * error, mean + 4 * error, 500)
    if ci_type == "known":
         y = norm.pdf(x, mean, error)
         label = "Distribución Normal"
    else:
         y = t.pdf(x, n-1, mean, error)
         label = f"Distribución t (df={n-1})"
    
    plt.figure(figsize=(10,6))
    plt.plot(x, y, label=label)
    
    # Resaltando el intervalo de confianza
    ix = np.linspace(lower, upper, 200)
    if ci_type == "known":
         iy = norm.pdf(ix, mean, error)
    else:
         iy = t.pdf(ix, n-1, mean, error)
    
    plt.fill_between(ix, iy, color='skyblue', alpha=0.5, label="Intervalo de Confianza")
    plt.axvline(mean, color='red', linestyle='--', label="Media Muestral")
    plt.axvline(lower, color='green', linestyle='--', label="Límite Inferior")
    plt.axvline(upper, color='green', linestyle='--', label="Límite Superior")
    plt.title("Distribución de la Media y su Intervalo de Confianza")
    plt.xlabel("Valor de la Media")
    plt.ylabel("Densidad de Probabilidad")
    plt.legend()
    plt.show()


## Ejercicios Interactivos y Reflexiones

Ahora que ya conoces la teoría y cómo se calcula un intervalo de confianza, te invito a explorar los siguientes aspectos interactivamente:

1. **Influencia del Tamaño de la Muestra (\(n\)):**
   - Modifica el valor de $n$ y observa cómo se modifica el ancho del intervalo.
   - **Reflexión:** ¿Por qué un tamaño de muestra mayor produce un intervalo más estrecho?

2. **Efecto del Nivel de Confianza:**
   - Cambia el nivel de confianza (por ejemplo, prueba con 90%, 95% y 99%) y analiza la variación en el intervalo.
   - **Reflexión:** A mayor nivel de confianza, ¿qué sucede con el valor crítico y el ancho del intervalo?

3. **Comparación Entre Métodos:**
   - Alterna entre “sigma conocido” y “sigma desconocido” y observa las diferencias en los valores críticos y en el resultado.
   - **Reflexión:** ¿En qué situaciones resulta más adecuado cada método?

Experimenta con los controles interactivos y analiza cómo cada parámetro afecta el cálculo del intervalo.


In [4]:
def interactive_session():
    # Widgets para seleccionar el método y parámetros
    ci_type_widget = widgets.RadioButtons(
         options=['sigma conocido', 'sigma desconocido'],
         description='Tipo:',
         disabled=False
    )
    sample_mean_widget = widgets.FloatText(
         description="Media Muestral:",
         value=100.0
    )
    sample_std_widget = widgets.FloatText(
         description="Desviación (\(\sigma\) o \(s\)):",
         value=15.0
    )
    n_widget = widgets.IntText(
         description="Tamaño n:",
         value=30
    )
    conf_level_widget = widgets.FloatSlider(
         description="Nivel de Confianza (%):",
         value=95.0,
         min=50,
         max=99.9,
         step=0.1,
         continuous_update=False
    )
    
    def update_ci(ci_type, sample_mean, sample_std, n, conf_level):
         # Convertir nivel de confianza a alfa
         alpha = 1 - conf_level/100
         if ci_type == 'sigma conocido':
             se = sample_std / np.sqrt(n)
             lower, upper, cv = calculate_ci(sample_mean, se, alpha, ci_type="known")
             method = "Distribución Normal (σ conocido)"
         else:
             se = sample_std / np.sqrt(n)
             lower, upper, cv = calculate_ci(sample_mean, se, alpha, ci_type="unknown", n=n)
             method = f"Distribución t (df = {n-1}, σ desconocido)"
         
         result_md = f"""
### Resultado del Cálculo

**Método:** {method}  
**Media Muestral:** {sample_mean:.2f}  
**Error Estándar:** {se:.2f}  
**Valor Crítico:** {cv:.2f}  
**Intervalo de Confianza al {conf_level}%:** [{lower:.2f}, {upper:.2f}]
         """
         
         clear_output(wait=True)
         display(Markdown(result_md))
         plot_ci(sample_mean, se, lower, upper, cv, ci_type=("known" if ci_type=='sigma conocido' else "unknown"), n=n)
    
    ui = widgets.VBox([ci_type_widget, sample_mean_widget, sample_std_widget, n_widget, conf_level_widget])
    out = widgets.interactive_output(update_ci, {
        'ci_type': ci_type_widget,
        'sample_mean': sample_mean_widget,
        'sample_std': sample_std_widget,
        'n': n_widget,
        'conf_level': conf_level_widget
    })
    display(ui, out)

# Ejecutamos la sesión interactiva
interactive_session()


<>:13: SyntaxWarning: invalid escape sequence '\('
<>:13: SyntaxWarning: invalid escape sequence '\('
D:\Usuarios\jcastebe\AppData\Local\Temp\ipykernel_17756\718986595.py:13: SyntaxWarning: invalid escape sequence '\('
  description="Desviación (\(\sigma\) o \(s\)):",


Output()

## Conclusiones y Reflexiones Finales

La sesión interactiva nos ha permitido:
- Comprender en profundidad cómo calcular intervalos de confianza para la media.
- Visualizar gráficamente cómo varían los intervalos según el tamaño de la muestra, el nivel de confianza y la variabilidad.
- Reflexionar sobre la aplicabilidad práctica de estos conceptos en la toma de decisiones en industrias, investigación y análisis de datos.

Te invito a experimentar con los controles y a discutir con tus compañeros sobre qué escenarios reales pueden beneficiarse de este tipo de análisis estadístico.
